In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY

import mysql.connector
from mysql.connector import errorcode
from ittutils import ittconnection, get_resampled, get_raw_price, get_raw_volume, classification_dataset_from_ts

In [2]:
db_connection = ittconnection('prodcopy')

cursor = db_connection.cursor()


In [10]:
query = (" SELECT * FROM signal_signal WHERE `signal` = 'RSI' and resample_period = 240")
#query = (" SELECT * FROM signal_signal WHERE (`signal` = 'ANN_Simple' or `signal` = 'RSI') and resample_period = 1440")

signals_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
signals_df['timestamp'] = pd.to_datetime(signals_df['timestamp'], unit='s')
signals_df.index = pd.DatetimeIndex(signals_df.timestamp)

signals_df.sort_index(inplace=True)

#signals_df[['timestamp','id','source','signal','horizon','resample_period']]

signals_df.tail(10)



,id,created_at,modified_at,UI,subscribers_only,text,source,transaction_currency,signal,trend,...,volume_usdt_change,sent_at,timestamp,rsi_value,counter_currency,resample_period,predicted_ahead_for,probability_down,probability_same,probability_up
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-05-14 16:00:00,241536,2018-05-14 16:06:19.088955,2018-05-14 16:06:19.757953,0,1,,2,ADA,RSI,1,...,None,1.526314e+09,2018-05-14 16:00:00,29.570480,1,240,None,None,None,None
2018-05-14 16:00:00,241526,2018-05-14 16:05:57.451528,2018-05-14 16:05:57.916375,0,1,,1,GRS,RSI,-1,...,None,1.526314e+09,2018-05-14 16:00:00,77.239472,0,240,None,None,None,None
2018-05-14 16:00:00,241521,2018-05-14 16:05:45.926846,2018-05-14 16:05:46.148221,0,1,,2,AST,RSI,1,...,None,1.526314e+09,2018-05-14 16:00:00,28.561026,1,240,None,None,None,None
2018-05-14 16:00:00,241512,2018-05-14 16:05:08.753914,2018-05-14 16:05:09.173023,0,1,,1,RLC,RSI,-1,...,None,1.526314e+09,2018-05-14 16:00:00,74.388528,0,240,None,None,None,None
2018-05-14 16:00:00,241508,2018-05-14 16:05:04.064219,2018-05-14 16:05:04.372835,0,1,,2,REP,RSI,1,...,None,1.526314e+09,2018-05-14 16:00:00,7.064653,1,240,None,None,None,None
2018-05-14 16:00:00,241481,2018-05-14 16:03:53.728765,2018-05-14 16:03:54.200215,0,1,,2,RLC,RSI,-1,...,None,1.526314e+09,2018-05-14 16:00:00,75.264589,0,240,None,None,None,None
2018-05-14 16:00:00,241460,2018-05-14 16:03:12.851483,2018-05-14 16:03:13.159134,0,1,,0,XEM,RSI,-1,...,None,1.526314e+09,2018-05-14 16:00:00,71.956041,0,240,None,None,None,None
2018-05-14 16:00:00,241952,2018-05-14 16:27:42.166540,2018-05-14 16:27:42.610759,0,1,,2,GRS,RSI,-1,...,None,1.526315e+09,2018-05-14 16:00:00,74.026740,1,240,None,None,None,None
2018-05-14 16:00:00,241693,2018-05-14 16:13:48.683641,2018-05-14 16:13:48.932282,0,1,,1,BLT,RSI,1,...,None,1.526314e+09,2018-05-14 16:00:00,29.504575,1,240,None,None,None,None


In [8]:
query = (" SELECT * FROM indicator_annpriceclassification WHERE resample_period=1440 AND transaction_currency='QWARK' ;")

signals_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
signals_df['timestamp'] = pd.to_datetime(signals_df['timestamp'], unit='s')
signals_df.index = pd.DatetimeIndex(signals_df.timestamp)

signals_df.sort_index(inplace=True)

#signals_df[['timestamp','id','source','signal','horizon','resample_period']]

signals_df.tail(10)

,id,source,counter_currency,transaction_currency,timestamp,resample_period,predicted_ahead_for,probability_same,probability_up,probability_down,ann_model_id
timestamp,,,,,,,,,,,
2018-05-11,29407,1,0,QWARK,2018-05-11,1440,900,0.406479,0.231834,0.361687,1
2018-05-12,48000,1,0,QWARK,2018-05-12,1440,900,0.437663,0.291604,0.270734,1
2018-05-13,66659,1,0,QWARK,2018-05-13,1440,900,0.378585,0.310117,0.311298,1
2018-05-14,84265,1,0,QWARK,2018-05-14,1440,900,0.350564,0.348365,0.301071,1
